In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[2], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation= 'selu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [8]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 10s 177ms/step - loss: 0.6808 - accuracy: 0.5608
Epoch 2/30
58/58 [==============================] - 10s 176ms/step - loss: 0.5543 - accuracy: 0.7262
Epoch 3/30
58/58 [==============================] - 10s 179ms/step - loss: 0.5236 - accuracy: 0.7492
Epoch 4/30
58/58 [==============================] - 10s 178ms/step - loss: 0.4676 - accuracy: 0.7744
Epoch 5/30
58/58 [==============================] - 11s 182ms/step - loss: 0.4800 - accuracy: 0.7853
Epoch 6/30
58/58 [==============================] - 10s 180ms/step - loss: 0.3985 - accuracy: 0.8269
Epoch 7/30
58/58 [==============================] - 11s 182ms/step - loss: 0.3117 - accuracy: 0.8817
Epoch 8/30
58/58 [==============================] - 11s 183ms/step - loss: 0.3047 - accuracy: 0.8959
Epoch 9/30
58/58 [==============================] - 11s 186ms/step - loss: 0.2787 - accuracy: 0.8894
Epoch 10/30
58/58 [==============================] - 11s 188ms/step - loss: 0.2694 - accura

58/58 [==============================] - 10s 173ms/step - loss: 0.2714 - accuracy: 0.8992
Epoch 22/30
58/58 [==============================] - 10s 172ms/step - loss: 0.2287 - accuracy: 0.9200
Epoch 23/30
58/58 [==============================] - 10s 174ms/step - loss: 0.2433 - accuracy: 0.9135
Epoch 24/30
58/58 [==============================] - 10s 172ms/step - loss: 0.2815 - accuracy: 0.9080
Epoch 25/30
58/58 [==============================] - 10s 173ms/step - loss: 0.6591 - accuracy: 0.6889
Epoch 26/30
58/58 [==============================] - 10s 172ms/step - loss: 0.5789 - accuracy: 0.6857
Epoch 27/30
58/58 [==============================] - 10s 169ms/step - loss: 0.5456 - accuracy: 0.7251
Epoch 28/30
58/58 [==============================] - 10s 172ms/step - loss: 0.4749 - accuracy: 0.7689
Epoch 29/30
58/58 [==============================] - 10s 172ms/step - loss: 0.4020 - accuracy: 0.8204
Epoch 30/30
58/58 [==============================] - 10s 173ms/step - loss: 0.4135 - accuracy:

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation= 'selu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [12]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [13]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 18s 315ms/step - loss: 0.6666 - accuracy: 0.5728
Epoch 2/30
58/58 [==============================] - 18s 318ms/step - loss: 0.5695 - accuracy: 0.6999
Epoch 3/30
58/58 [==============================] - 18s 317ms/step - loss: 0.4488 - accuracy: 0.8072
Epoch 4/30
58/58 [==============================] - 18s 318ms/step - loss: 0.5575 - accuracy: 0.7032
Epoch 5/30
58/58 [==============================] - 19s 320ms/step - loss: 0.6035 - accuracy: 0.6440
Epoch 6/30
58/58 [==============================] - 18s 317ms/step - loss: 0.4475 - accuracy: 0.8269
Epoch 7/30
58/58 [==============================] - 18s 316ms/step - loss: 0.3650 - accuracy: 0.8576
Epoch 8/30
58/58 [==============================] - 18s 317ms/step - loss: 0.4760 - accuracy: 0.7996
Epoch 9/30
58/58 [==============================] - 18s 315ms/step - loss: 0.5964 - accuracy: 0.6670
Epoch 10/30
58/58 [==============================] - 19s 319ms/step - loss: 0.5387 - accura

58/58 [==============================] - 17s 296ms/step - loss: 0.1695 - accuracy: 0.9409
Epoch 22/30
58/58 [==============================] - 17s 296ms/step - loss: 0.2801 - accuracy: 0.8828
Epoch 23/30
58/58 [==============================] - 17s 295ms/step - loss: 0.1982 - accuracy: 0.9211
Epoch 24/30
58/58 [==============================] - 17s 297ms/step - loss: 0.1879 - accuracy: 0.9441
Epoch 25/30
58/58 [==============================] - 17s 292ms/step - loss: 0.6967 - accuracy: 0.6670
Epoch 26/30
58/58 [==============================] - 17s 296ms/step - loss: 0.4993 - accuracy: 0.7470
Epoch 27/30
58/58 [==============================] - 17s 295ms/step - loss: 0.5288 - accuracy: 0.7338
Epoch 28/30
58/58 [==============================] - 17s 297ms/step - loss: 0.4514 - accuracy: 0.7908
Epoch 29/30
58/58 [==============================] - 17s 295ms/step - loss: 0.3556 - accuracy: 0.8510
Epoch 30/30
58/58 [==============================] - 17s 294ms/step - loss: 0.3426 - accuracy: